# Install and import libraries

In [ ]:
!pip install einops
!pip install datasets
!pip install torch-fidelity
!pip install torchmetrics
!pip3 install image-similarity-measures

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 508 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 11.8 MB/s 
     |████████████████████████████████| 182 kB 12.3 MB/s 
     |████████████████████████████████| 115 kB 52.3 MB/s 
     |████████████████████████████████| 212 kB 35.2 MB/s 
     |████████████████████████████████| 127 kB 36.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 14.9 MB/s 
Looking in indexes: https://pypi.org/simp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import numpy
from numpy import cov, trace, iscomplexobj
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm

import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import sys

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Paths (Checkpoint)

In [ ]:
FashionCNN_checkpoint = "/content/drive/MyDrive/Colab Notebooks/DiffusionModels/fashion_mnist_CNNClassifier_91Acc.pt"
DiffusionModel_checkpoint = "/content/drive/MyDrive/Colab Notebooks/DiffusionModels/fashion_mnist/checkpoint_T300_cosine/fashion_mnist/Unet.ckpt"

Stats_path = "/content/drive/MyDrive/Colab Notebooks/DiffusionModels/fashion_mnist/checkpoint_T300_cosine/fashion_mnist/stats_T300_cosine.csv"

# Load FashionMNIST Dataset to get Real Images

In [ ]:
train_set = torchvision.datasets.FashionMNIST("./data", download=True, transform=
                                                transforms.Compose([transforms.ToTensor()]))
train_loader = torch.utils.data.DataLoader(train_set, 
                                           batch_size=100, shuffle=False)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



# Generate Images from trained Diffusion Model

In [ ]:
!git clone https://github.com/GangaMegha/Diffusion-Models.git 

Cloning into 'Diffusion-Models'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 173 (delta 95), reused 127 (delta 53), pack-reused 0
Receiving objects: 100% (173/173), 29.34 KiB | 9.78 MiB/s, done.
Resolving deltas: 100% (95/95), done.


In [ ]:
%cd /content/Diffusion-Models/src
!ls

/content/Diffusion-Models/src
config.py	diffusion  main.py  train
data_loader.py	logger.py  models   visualize


#### Load Model

In [ ]:

from models.unet import Unet as Model
from config import LEVEL, TRAIN, MODEL, MODEL_TYPE, DATASET_NAME, CONFIG, CHECKPOINT_PATH, RESULT_PATH, LOG_PATH

from diffusion.reverse_data_generate import sample
from diffusion.var_schedule import alpha_beta

from data_loader import reverse_transform

cfg = CONFIG[MODEL_TYPE.value][DATASET_NAME]
Noise_model = Model(cfg)
Noise_model_checkpoint = torch.load(DiffusionModel_checkpoint)
Noise_model.load_state_dict(Noise_model_checkpoint['state_dict'])
Noise_model.to(device)
Noise_model.eval()

variance_dict = alpha_beta(cfg.get('T'), schedule=cfg.get("var_schedule"))

#### Generate Images

In [ ]:
fake_images = []
for i in range(10):
  fake_images_list = sample(Noise_model, variance_dict, cfg, sample_cnt=100, every50=False)
  fake_images.append(reverse_transform()(torch.cat(fake_images_list, dim=0)))

sampling loop time step:   0%|          | 0/300 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/300 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/300 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/300 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/300 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/300 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/300 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/300 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/300 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/300 [00:00<?, ?it/s]

# Get FashionCNN results for Real and Fake Images

In [ ]:
class FashionCNN(nn.Module):
    
    def __init__(self):
        super(FashionCNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding="same"),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding="same"),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(in_features=64*7*7, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = nn.ReLU()(self.fc1(out))
        out = nn.Softmax()(self.fc2(out))
        
        return out

FashionCNN_model = FashionCNN()
CNN_checkpoint = torch.load(FashionCNN_checkpoint)
FashionCNN_model.load_state_dict(CNN_checkpoint['state_dict'])
FashionCNN_model.to(device)
FashionCNN_model.eval()

FashionCNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.clone().detach()
    return hook

FashionCNN_model.fc1.register_forward_hook(get_activation('fc1'))

#### Real Images

In [ ]:
real_out = []
output_real = []
it = iter(train_loader)
batches = []
for i in range(10):
  batch = next(it)
  batches.append(batch[0])
  output_real.append(FashionCNN_model(batch[0].to(device)))
  real_out.append(activation['fc1'].detach().cpu().numpy())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


#### Fake Images

In [ ]:
fake_out = []
output_fake = []
for i in range(10):
  output_fake.append(FashionCNN_model((fake_images[i].to(dtype=torch.float)/255.0).to(device)))
  fake_out.append(activation['fc1'].detach().cpu().numpy())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


## Calculate FID

In [ ]:
# Ref : https://machinelearningmastery.com/how-to-implement-the-frechet-inception-distance-fid-from-scratch/
def calculate_fid(act1, act2):
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

In [ ]:
fid_cnn_list = []
for i in range(10):
  fid_cnn_list.append(calculate_fid(real_out[i], fake_out[i]))

print("fid_cnn_list : ", fid_cnn_list)
print("Mean : ", np.mean(np.array(fid_cnn_list)))
print("Std : ", np.std(np.array(fid_cnn_list)))

fid_cnn_list :  [77625.70039924371, 80734.68350439571, 90541.34298659414, 78703.89784471835, 74731.19781611126, 106765.3685553512, 96441.77772172155, 73677.83179966557, 66162.40043372384, 83442.2095626946]
Mean :  82882.64106242199
Std :  11373.898717883752


## Calculate Inception Score using CNN Classifier Probabilities

In [ ]:
# Ref : https://machinelearningmastery.com/how-to-implement-the-inception-score-from-scratch-for-evaluating-generated-images/
# calculate the inception score for p(y|x)
def calculate_inception_score(p_yx, eps=1E-16):
	# calculate p(y)
	p_y = np.expand_dims(p_yx.mean(axis=0), 0)
	# kl divergence for each image
	kl_d = p_yx * (np.log(p_yx + eps) - np.log(p_y + eps))
	# sum over classes
	sum_kl_d = kl_d.sum(axis=1)
	# average over images
	avg_kl_d = np.mean(sum_kl_d)
	# undo the logs
	is_score = np.exp(avg_kl_d)
	return is_score

In [ ]:
is_cnn_list = []
for i in range(10):
  is_cnn_list.append(calculate_inception_score(output_fake[i].detach().cpu().numpy()))

print("is_cnn_list : ", is_cnn_list)
print("Mean : ", np.mean(np.array(is_cnn_list)))
print("Std : ", np.std(np.array(is_cnn_list)))

is_cnn_list :  [7.7958417, 7.340661, 8.034454, 7.6593637, 6.904479, 7.39914, 7.324825, 7.323767, 7.761414, 7.3812485]
Mean :  7.4925194
Std :  0.30535084


## Calculate SSIM and RMSE

In [ ]:
from image_similarity_measures.quality_metrics import rmse, ssim, sre, uiq

ssim_list_class = []
rmse_list_class = []

for label in range(10):
  ssim_list = []
  rmse_list = []

  for idx in range(10):
    predicted_real = torch.max(output_real[idx], 1)[1]
    predicted_fake = torch.max(output_fake[idx], 1)[1]

    real_img = []
    fake_img = []
    for j in range(len(predicted_real)):
      if predicted_real[j]==label:
        real_img.append(batches[idx][j])
    for j in range(len(predicted_fake)):
      if predicted_fake[j]==label:
        fake_img.append((fake_images[idx][j].to(dtype=torch.float)/255.0).to(device))

    for j in range(len(real_img)):
      for k in range(len(fake_img)):
        ssim_list.append(ssim(real_img[j].permute(1, 2, 0).cpu().numpy(), fake_img[k].permute(1, 2, 0).cpu().numpy()))
        rmse_list.append(rmse(real_img[j].permute(1, 2, 0).cpu().numpy(), fake_img[k].permute(1, 2, 0).cpu().numpy()))

  ssim_list_class.append(np.mean(np.array(ssim_list)))
  rmse_list_class.append(np.mean(np.array(rmse_list)))

print("ssim_list_class : ", ssim_list_class)
print("rmse_list_class : ", rmse_list_class)

print("ssim Mean : ", np.mean(np.array(ssim_list_class)))
print("ssim Std : ", np.std(np.array(ssim_list_class)))

print("rmse Mean : ", np.mean(np.array(rmse_list_class)))
print("rmse Std : ", np.std(np.array(rmse_list_class)))

ssim_list_class :  [0.999921209954663, 0.9999730827433083, 0.99989952794147, 0.9999449786172925, 0.9999560020359304, 0.9999749130115475, 0.9999050180610349, 0.9999726623614537, 0.999897061116258, 0.9999581990004187]
rmse_list_class :  [9.251046e-05, 6.914813e-05, 0.00010289707, 8.084915e-05, 8.168524e-05, 7.287776e-05, 0.0001001886, 6.7455316e-05, 0.00011029073, 8.5514526e-05]
ssim Mean :  0.9999402654843378
ssim Std :  3.0083390488782846e-05
rmse Mean :  8.634169e-05
rmse Std :  1.4012685e-05


# Compute True FID and IS using Inception Network

In [ ]:
from train.metrics import FID, IS

In [ ]:
FID_ = FID().to(device)
IS_ = IS().to(device)

Downloading: "https://github.com/toshas/torch-fidelity/releases/download/v0.2.0/weights-inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/weights-inception-2015-12-05-6726825d.pth


  0%|          | 0.00/91.2M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `InceptionScore` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [ ]:
it = iter(train_loader)
FID_list, IS_list = [], []

for i in range(10):
  batch = next(it)[0]
  FID_list.append(FID_(fake_images[i].to(device), (batch*255).to(dtype=torch.uint8).to(device), cfg["grayscale"]).item())
  IS_list.append(IS_(fake_images[i].to(device), cfg["grayscale"]).item())

print("FID_list : ", FID_list)
print("IS_list : ", IS_list)

print("FID Mean : ", np.mean(np.array(FID_list)))
print("FID Std : ", np.std(np.array(FID_list)))

print("IS Mean : ", np.mean(np.array(IS_list)))
print("IS Std : ", np.std(np.array(IS_list)))

FID_list :  [126.53873443603516, 129.41909790039062, 135.76522827148438, 118.53218078613281, 119.57352447509766, 137.77362060546875, 132.33824157714844, 122.90753173828125, 127.0515365600586, 122.74620819091797]
IS_list :  [2.7412374019622803, 2.704155683517456, 2.7559709548950195, 2.6248011589050293, 2.7558281421661377, 2.9869632720947266, 2.7194199562072754, 2.6724908351898193, 2.769301176071167, 2.977093458175659]
FID Mean :  127.26459045410157
FID Std :  6.2326869681694
IS Mean :  2.770726203918457
IS Std :  0.11343080477458697


# Save Computed stats

In [ ]:
# load csv module
import csv

# define a dictionary with key value pairs
diction = {
    "Metric" : "Value",
    "FID Mean" : np.mean(np.array(FID_list)),
    "FID Std" : np.std(np.array(FID_list)),
    "IS Mean" : np.mean(np.array(IS_list)),
    "IS Std" : np.std(np.array(IS_list)),
    "SSIM Mean" : np.mean(np.array(ssim_list_class)),
    "SSIM Std" : np.std(np.array(ssim_list_class)),
    "RMSE Mean" : np.mean(np.array(rmse_list_class)),
    "RMSE Std" : np.std(np.array(rmse_list_class)),
    "FID CNN Mean" : np.mean(np.array(fid_cnn_list)),
    "FID CNN Std" : np.std(np.array(fid_cnn_list)),
    "IS CNN Mean" : np.mean(np.array(is_cnn_list)),
    "IS CNN Std" : np.std(np.array(is_cnn_list))
    }

# open file for writing, "w" is writing
w = csv.writer(open(Stats_path, "w"))

# loop over dictionary keys and values
for key, val in diction.items():

    # write every key and value to file
    w.writerow([key, val])
